# **DATA CRAWLING WITH PYTHON**
<BR><BR>

## **SET UP ENVIRONMENT**

In [ ]:
%pip install cloudscraper                   
%pip install playwright
%pip install beautifulsoup4             
%pip install pandas 
%pip install openpyxl 
%pip install chromedriver-autoinstaller
%pip install lxml
%pip install PyYAML
%pip install requests
%pip install numpy
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## **CRAWLING DATA**

---
### **Import Libraries**

In [ ]:
import os
import json                                 # Handle JSON data
import time                                 # Create latencies between 2 requests to avoid being blocked
import re                                   # Use regular expression( biểu thức chính quy) to processing strings

import requests                             # Send HTTP requests to fetch web content
from bs4 import BeautifulSoup               # Parse HTTP and extract information
from tqdm import tqdm                       # Display progress bar

import yaml                                 # Read configuration files
from typing import Dict, List               # Provide type hints for cleaner code
from urllib.parse import urljoin, urlparse # Handle urls
import random

---

### **Connect the website and receive HTML response**

For this step, we have to define functions to send requests to website and receive HTML response. However, there are some barriers of website preventing the requests to avoid automatically crawling by bots, which can make server overload. The functions should overcome this limitation to crawl data politely.

**Sending HTTP requests**

A HTTP request contains 3 parts:
* *`Request Line`*: includes:
    * Method: `GET` to request data, `POST` to send data
    * URI: Path to target resource
    * HTTP Version

* *`Heading`*: includes additional information which are type of data, language, where the data comes from, verification information,...

* *`Body`*: includes content that users want to send to server( from forms or files attached)

The anti-detection data crawling technique will create a HTTP request that looks like coming from normal groups of users but not from a bot.

**Anti-detection Techniques**

* ***User-agent rotation***: rotate User-Agent string to mimic different browser( thay đổi User-Agent để bắt chước các trình duyệt khác):
    * Field User-agent in the Header part of a HTTP request indicates the config of the hardware sending messages

* ***Browser-like headers***: Use headers that resemble a real browser( sử dụng headers giống trình duyệt thực)

* ***Random delay***: Randomly delay between 2 requests

* ***Retry mechanism***: Retry up to 3 times on failure

In [ ]:
def get_html(url, domain_name = 'https://batdongsan.com.vn/'):
    '''
    Function gets the HTML document from an url with anti-detection techniques
    '''
    import random
    import requests

    if not url.startswith('http'):
        return None 
    
    # list of user-agent that mimics the behaviors of real users
    user_agents = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    ]

    # create headers which simulate the real header, randomly using the user-agents created
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-platform': '"macOS"',
        'User-Agent': random.choice(user_agents),
        'Referer': domain_name,
        'Origin': domain_name
    }

    # Create a new session to maintain cookies and headers accross multiple requests
    session = requests.Session()
    
    # Update session's header by above defined header
    session.headers.update(headers)
    
    # Try to request up to 3 times in case of failure
    for attempt in range(3):
        try:
            time.sleep(random.uniform(1,3))

            response = session.get(
                url,
                timeout=30,
                allow_redirects=True,
                stream=False
            )

            if response.status_code == 200:
                response.encoding = 'utf-8' # using utf-8 for encoding Vietnamese
                html_text = response.text
                response.close()
                session.close()
                return html_text
            
            # If request is denined
            elif response.status_code == 403:
                # Try to change user-agent and retry
                headers['User-Agent'] = random.choice(user_agents)
                session.headers.update(headers)
                time.sleep(random.uniform(2,5))                     # Wait for 2-5 seconds
                continue

            else:
                if attempt == 2:
                    print("Request to {} is deninded.".format(url))
                response.close()

        # If SSL Error --> Try again
        except requests.exceptions.SSLError:
            continue

        # If the request does not receive any responses due to general errors: time out, connection fail,...
        except requests.exceptions.RequestException as e:
            if attempt == 2:
                print("Request error for {}: {}".format(url, e))
        
        # Catch other exception else
        except Exception as e:
            if attempt == 2:
                print("Unknown error for {}: {}".format(url, e))

        # Increase delay time between retries to avoid being blocked by server
        if attempt < 2:
            time.sleep(random.uniform( 3 + attempt, 6 + attempt))

    session.close()     # end sesion
    return None

**Advanced anti-detection techniques**

Sometimes, websites may block the Python requests but still allow access from other tools such as `cURL`. These tools are similar with `requests` of Python, they help to send requests and receive response from webs. Therefore, we need a backup method in case of Python requests sent by function `get_html` being blocked.

`cURL` is a CLI tools, which only provides instructions on CLI instead of a library as `requests`.

In [ ]:
def get_html_curl(url, domain_name = 'https://batdongsan.com.vn/'):
    '''
    In case of Python requests is blocked by website,
    This function will try to send requests by cURL instead of Python requests
    '''
    import subprocess # Use to run cURL in code, which only provides instruction through CLI
    import tempfile   # Use to create temporary files
    import os

    if not url.startswith('http'):
        return None
    
    try:
        # Create a temporary file to store the downloaded HTML
        with tempfile.NamedTemporaryFile(mode = 'w+', delete=False, suffix='.html') as temp_file:
            temp_filename = temp_file.name

        # Build cURL to simulate a read browser
        curl_command = [
            'curl',
            '-s',
            '-L',
            '--compressed',
            '--max-time', '30',
            '--retry', '2',
             '--user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            # Use a realistic User-Agent to avoid detection (Giả lập trình duyệt thật để tránh bị chặn)

            # Common browser headers (Các header phổ biến mà trình duyệt gửi kèm)
            '--header', 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
            '--header', 'Accept-Language: vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7',
            '--header', 'Accept-Encoding: gzip, deflate, br',
            '--header', 'DNT: 1',  # Do Not Track header (Yêu cầu không theo dõi)
            '--header', 'Connection: keep-alive',
            '--header', 'Upgrade-Insecure-Requests: 1',
            '--header', 'Sec-Fetch-Dest: document',
            '--header', 'Sec-Fetch-Mode: navigate',
            '--header', 'Sec-Fetch-Site: none',
            '--header', 'Sec-Fetch-User: ?1',
            '--header', 'Cache-Control: max-age=0',
            '--referer', domain_name,  # Pretend to come from vnexpress homepage (Giả vờ người dùng click từ trang chủ)
            '--output', temp_filename,              # Save output HTML to temp file (Ghi nội dung tải về vào file tạm)
            url
        ]

        result = subprocess.run(
            curl_command,
            capture_output = True,
            text = True, 
            timeout = 35
        )

        # If sucessfully receive response: response is saved in temp_filename
        if result.returncode == 0:
            # Extract content saved in temporary file
            with open(temp_filename, 'r', encoding = 'utf-8', errors='ignore') as f:
                content = f.read()

            # Remove the temp file after reading
            os.unlink(temp_filename)

            # Check if the content is long enough
            if len(content) > 3000:
                return content
        
        # If temp file still exists, delete it
        if os.path.exists(temp_filename):
            os.unlink(temp_filename)

    except Exception as e:
        print("Curl method failed for {}: {}".format(url, e))

    return None

**Other Advanced anti-detection technique**

Sometimes, the website uses a waiting page to verify the real action of browsers. When the website receives a request, a middle service like Cloudflare is used to bring a challenge to verify before redirect to the main website. For this challenge, we will use `playwright` to control the browser to pass the challenge. 

In [ ]:
'''
    install chrome driver 
    this driver is an embedded browser containing fully apis of chrome
    this is independent with the browser chrome in local computer
    the library playwright will use the chrominum embeded instead of the browser in local
''' 
%playwright install chrominum 

In [ ]:
from playwright.sync_api import sync_playwright
import random, time

class AccessWebsite():
    def __init__(self, page, browser):
        self.page = page
        self.behaviours_ls = [
            self.human_delay,
            self.human_move,
            self.human_scroll,
            self.human_wait_timeout,
            self.human_random_click
        ]

    def human_delay(self, start=1.0, end=3.0):
        '''
        Method to mimic the human delay, which is usually random
        '''
        time.sleep(random.uniform(start, end))

    def human_move(self):
        '''
        Method to mimic the mouse moving
        '''
        self.page.mouse.move(random.randint(100,400), random.randint(100,400))
    
    def human_scroll(self):
        '''
        Method to mimic the mouse scrolling
        '''
        length_scroll = random.randint(100, 500)
        self.page.wheel(0, length_scroll)

    def human_wait_timeout(self):
        '''
        Method to mimic the time waiting
        '''
        self.page.wait_for_timeout(random.randint(300, 1200))

    def human_random_click(self):
        '''
        Method to mimic the random click
        '''
        x, y = random.randint(100, 900), random.randint(100, 600)
        self.page.mouse.click(x, y)

    def simulate_human_behaviours(self):
        '''
        Method to mimic the human behaviour in general
        '''
        num_of_choosen_behaviours = random.randint(1, 5)
        behaviours_ls = []
        for _ in range(num_of_choosen_behaviours):
            rand_behaviour = random.randint(0,4)
            behaviours_ls.append(self.behaviours_ls[rand_behaviour])

        for method in behaviours_ls:
            method()

    def get_html(self, url:str):
        '''
        Method to access the url and get html document
        '''
        try:
            self.page.goto(url)
            self.simulate_human_behaviours()
            html_text = self.page.content()
            return html_text
        except Exception as e:
            print("!!! |X| Exception in using playwright to open browser: {}".format(e))
            return None

def open_browser():
    '''
    Open browser --> return browser and page_controller
    This should be called once
    '''
    try:
        with sync_playwright() as p:
            browser = p.chromium.launch(headless=False) # headless = False to hide browser window, True if show
            page = browser.new_page()
            return browser, page
    except Exception as e:
        print("!!! |X| Exception in using playwright to open browser: {}".format(e))
        return None

def close_browser(browser):
    '''
    Close browser
    '''
    browser.close()

Playwright for simulating the behaviour of using browser is sometimes heavy. There is an other library which is more efficient than Playwright, but only for passing Cloudflare: `cloudscaper`

In [ ]:
def get_html_pass_cloudflare(
        url: str, 
        state_file: str = 'cookies.json',
        domain_name = 'https://batdongsan.com.vn/') -> str:
    '''
    Function to get html document text --> Return: html text
    '''
    import cloudscraper, random, time, json, os, requests
    if not url.startswith('http'):
        return None 
    
    # list of user-agent that mimics the behaviors of real users
    user_agents = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    ]

    # create headers which simulate the real header, randomly using the user-agents created
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-platform': '"macOS"',
        'User-Agent': random.choice(user_agents),
        'Referer': domain_name,
        'Origin': domain_name
    }

    # Load cookies
    if state_file:
        with open(state_file, "r") as f:
            cookies = json.load(f)
            scraper.cookies.update(cookies)
    
    scraper = cloudscraper.create_scraper()
    # Try up to 3 times if response is not accepted
    for attempt in range(3):
        try:
            time.sleep(random.uniform(1,3))

            response = scraper.get(url, headers = headers, timeout = random.randint(100,300))

            # save cookies
            if state_file:
                with open(state_file, "w") as f:
                    json.dump(requests.utils.dict_from_cookiejar(scraper.cookies), f)
        
            # Check response status
            if response.status_code in (429, 503) or "just a moment" in response.text.lower() or response.text.lower():
                # Try to use other user-agent
                headers['User-Agent'] = random.choice(user_agents)
                time.sleep(random.uniform(2,5))
                continue
            
            elif response.status_code == 200:
                html_text = response.text
                return html_text
            
            elif attempt == 2:
                print("[|X| FAIL] Cloudflare block this request")
                return None

        except Exception as e:
            print("[!!!| EXCEPTION] Exception : {}".format(e))
            return None

In [ ]:
def get_html_combine(url, domain_name = 'https://batdongsan.com.vn/'):
    '''
    This function is responsible for getting html response from websites by combination of both above functions
    '''
    content = get_html(url, domain_name=domain_name)
    if content:
        return content
    
    print("Trying curl method for {}".format(url[:50]))
    content = get_html_curl(url, domain_name = domain_name)
    if content:
        print("Curl success!")
        return content
    
    # If both methods fail, return None
    return None


---
### **Parse HTTP document & Extract information**

This step is responsible for parsing http response, which can help more easily extract information from document.

**Parse HTTP document**

In [ ]:
def parse_html(html_text):
    '''
    This function works for parsing http document
    '''
    from bs4 import BeautifulSoup
    return BeautifulSoup(html_text, 'lxml')

**Extract information**

This step is responsible for extract information in a `BeautifulSoup` object, which is an object after parsing the http document. This is done by `select(selector:str)`. However, because the extraction depends on selectors, while the selectors are different with each website. In this document, I will choose articles on the website https://batdongsan.com.vn/tintuc.

In [ ]:
def get_title(html_parse: BeautifulSoup) -> list:
    '''
    Function gets the title of the articles after parsing to BeautifulSoup object
    --> Return : List of title
    '''
    return [html_parse.title.text]

def get_publisher(html_parse: BeautifulSoup) -> list:
    '''
    Function gets the publisher, or list of publisher of the document
    Publisher is a person or an organization publishing the article, not the author
    --> Return : List of publisher
    '''
    selector = '#__next main > div.ArticlePageTemplate_articlePageContainer__wcRoZ.container > div:nth-child(2) > div > div > div > div.AuthorInfo_authorName__m9KD3 > a'
    publisher_ls = html_parse.select(selector)
    if len(publisher_ls):
        return [publisher.text for publisher in publisher_ls]
    return None

def get_latest_update(html_parse: BeautifulSoup) -> list:
    '''
    Function gets the latest update time
    --> Return: list of update time
    '''
    selector = '#__next main > div.ArticlePageTemplate_articlePageContainer__wcRoZ.container > div:nth-child(2) > div > div > div > div.AuthorInfo_postDate__UTKIr'
    update_time_ls = html_parse.select(selector)

    if len(update_time_ls):
        return [update_time.text for update_time in update_time_ls]
    return None

def get_content(html_parse: BeautifulSoup) -> dict:
    '''
    Function gets the article's content
    --> Return: dict: {
        "content"       : str,
        "author"        : str,
        "source"        : str,
        "publish time"  : str,
        "link source"   : str
    }
    '''
    selector = '#__next main > div.ArticlePageTemplate_articlePageContainer__wcRoZ.container > div:nth-child(4) > div.col-xl-8.col-lg-8.col-md-12.col-12 > article > div:nth-child(1) > *'
    content_list = html_parse(selector)

    if len(content_list):
        full_content = ''
        accept_tag = ['p', 'h1', 'h2', 'h3', 'h4', 'h5'] # list of acceptable tags, this is for eleminating the figure
        end_main_content = ['——']                        # list of signals which indicate that the main content ends

        author = 'Unknown'
        source = 'Unknown'
        publish_time = 'Unknown'
        source_link = 'Unknown'

        # Extract full content, full content ends by the line '——'
        for i in range(len(content_list)):
            if content_list[i].name in accept_tag and content_list[i].text not in end_main_content:
                full_content += content_list[i].text + "\n\n"
        
        # Extract author and original article: 
        # after main content, the information of author and the original article is shown in (i+1)-th element
        # ( Because the i-th element is the line '——')
        if i+1 < len(content_list)-1: 
            # format of information is:
            # <p>Tác giả:...<br>Nguồn:...<br>
            # split it into separate parts: ['Tác giả:...', <br>, 'Nguồn:...', <br>,...] by method `children`
            for child in content_list[i+1].children:
                if child.name != "br":
                    separate_index = child.text.find(":")+1
                    info = child.text[separate_index:]
                    format_text = " ".join(info.split())
                    if 'tác giả' in child.text.lower():
                        author = format_text
                    if 'nguồn tin' in child.text.lower():
                        source = format_text
                    if 'thời gian' in child.text.lower():
                        publish_time = format_text
                    if 'link' in child.text.lower():
                        source_link = format_text

            return {
                "content": full_content,
                "author" : author,
                "source" : source,
                "publish time": publish_time,
                "source link": source_link
            }
        
    return None

**[FULL]: Getting information of a url link**

In [ ]:
def get_full_information(url: str) -> dict:
    '''
    Function gets full information of an article
    --> Return: dict:
        {
            "title"                 : str,
            "publisher"             : str,
            "lastest update time"   : str,
            "content"               : str,
            "author"                : str,
            "source"                : str,
            "publish time"          : str,
            "link source"           : str
        }
    '''
    print("\rCrawling {}".format(url), end = "")

    # Send request and receive html text
    html_text = get_html_combine(url)
    # parsing html
    html_parse = parse_html(html_text)
    # Get title
    print("\r       Getting title...", end = "")
    title = get_title(html_parse)
    if title:
        title = title[0]
    else: 
        title = 'Unknown'
        print("     |X| Cannot get the title !!!")
        
    # Get publisher
    print("\r       Getting publisher...", end = "")
    publiser = get_publisher(html_parse)
    if publiser:
        publiser = ", ".join(publiser)
    else:
        print(("      |X| Cannot get the publisher !!!"))
        publiser = 'Unknown'

    # Get latest update time
    print("\r       Getting latest update time...", end="")
    latest_update_time = get_latest_update(html_parse)
    if latest_update_time:
        latest_update_time = " ".join(latest_update_time)
    else:
        print(("    |X| Cannot get the latest update time !!!"))
        publiser = 'Unknown'

    # Get content, author, source, publish time, source link
    result = get_content(html_parse)
    print("\r       Getting content, author, source, publish time...", end = "")
    if result:
        result["title"] = title
        result["publisher"] = publiser
        result["latest update time"] = latest_update_time
        print("\r       Finish crawling!", end = "")
        
    else:
        print("    |X| Cannot get the content, author, source, publish time!!!")
        return {
            "title"                 : title,
            "publisher"             : publiser,
            "lastest update time"   : latest_update_time,
            "content"               : 'Unknown',
            "author"                : 'Unknown',
            "source"                : 'Unknown',
            "publish time"          : 'Unknown',
            "link source"           : 'Unknown'
        }
        
    return result

In [ ]:
'''
QUICK TEST
'''
test_url = 'https://batdongsan.com.vn/tin-tuc/maison-grand-buoc-ngoat-moi-tren-hanh-trinh-kien-tao-chuan-song-chuyen-gia-tai-sieu-cang-quoc-te-phu-my-847604'
result = get_full_information(test_url)
for key, value in result.items():
    print("{} : {}".format(key, value))